<a href="https://colab.research.google.com/github/vinzentp/Mission-Calypso/blob/master/Test_all_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import keras as keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from itertools import combinations


def show_plot_of_predicted_vs_true(X, y_pred, y_true):
  
  #X_flat = [feature for timestep in X for feature in timestep]

  plt.figure(figsize = (20,8))
  plt.plot(X, marker = '.')
  plt.plot(range(len(X), len(X)+len(y_pred)), y_pred, marker = 'x')
  plt.plot(range(len(X), len(X)+len(y_true)), y_true, marker = '.')

df = pd.read_csv('drive/MyDrive/KDD_Weather_Prediction/madrid.csv')
df['time'] = pd.to_datetime(df['time'])

df = df.sort_values('time')

# Shape definitions
n_steps = 50
y_pred_steps = 10
timesteps_per_batch = n_steps + y_pred_steps

# Calculating the number of batches
n_batches = len(df) // timesteps_per_batch

# Triming the data to a multiple of the batch size
df = df[:n_batches * timesteps_per_batch]

#column_names= ["temperature", "wind_speed", "wind_direction"]
column_names= ["temperature", "wind_speed", "wind_direction", "humidity", "barometric_pressure","solar_radiation", "precipitation"]


results_list = []
for i in range(1, len(column_names)+1):
  for feature_combinations in combinations(column_names, i):

    # Droping everything but the temperature
    data = df[["temperature"]+list(feature_combinations)]
    print(data)
    #df = df[['temperature']]

    # Reshaping the data into a 3D array
    data = data.values.reshape((n_batches, timesteps_per_batch, -1))

    # Converting the data to a numpy array
    data = np.array(data)

    # Defining split sizes
    train_size = int(n_batches * 0.7)
    validation_size = int(n_batches * 0.15)
    test_size = n_batches - train_size - validation_size

    scaler = StandardScaler()
    num_instances, num_time_steps, num_features = data.shape
    data = np.reshape(data, (-1, num_features))
    data = scaler.fit_transform(data)
    data = np.reshape(data, (num_instances, num_time_steps, num_features))


    # Splitting in train, validation and test set
    X_train, y_train = data[:train_size, :n_steps, 1:], data[:train_size, -y_pred_steps:, 0]
    X_valid, y_valid = data[train_size:train_size+validation_size, :n_steps, 1:], data[train_size:train_size+validation_size, -y_pred_steps:, 0]
    X_test, y_test = data[train_size+validation_size:, :n_steps, 1:], data[train_size+validation_size:, -y_pred_steps:, 0]


    Y = np.empty((len(data), n_steps, y_pred_steps)) 
    for step_ahead in range(1, y_pred_steps + 1):
      for i in range(len(data)):
        for j in range(n_steps):
          Y[i,j,step_ahead-1] = data[i, j+step_ahead, 0]

    y_train = Y[:train_size]
    y_valid = Y[train_size:train_size + validation_size]
    y_test = Y[train_size+validation_size:]

    model = keras.models.Sequential([
        keras.layers.SimpleRNN(20, return_sequences=True, input_shape=(None, len(feature_combinations))),
        keras.layers.SimpleRNN(20, return_sequences=True),
        keras.layers.SimpleRNN(20, return_sequences=True),
        keras.layers.TimeDistributed(keras.layers.Dense(y_pred_steps))
    ])


    def last_time_step_mse(Y_true, Y_pred):
      return keras.metrics.mean_squared_error(Y_true[:,-1], Y_pred[:, -1])

    # Early Stopping Callback
    early_stop = EarlyStopping(monitor='val_loss', patience=200)

    model.compile(loss='mse', optimizer='adam', metrics=[last_time_step_mse])
    model.fit(X_train, y_train, epochs= 2000, validation_data=(X_valid, y_valid), callbacks=[early_stop])

    y_pred = model.predict(X_test)
    results_list.append("Run: "+ str(feature_combinations)+ ": "+ str(np.array(last_time_step_mse(y_pred, y_test)).mean()))
    print("Run: "+ str(feature_combinations)+"\n----------------\n" + str(results_list))



Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
10/10 [==============================] - 1s 61ms/step - loss: 0.0496 - last_time_step_mse: 0.0409 - val_loss: 0.0662 - val_last_time_step_mse: 0.0571
Epoch 257/2000
10/10 [==============================] - 1s 53ms/step - loss: 0.0523 - last_time_step_mse: 0.0453 - val_loss: 0.0714 - val_last_time_step_mse: 0.0628
Epoch 258/2000
10/10 [==============================] - 1s 52ms/step - loss: 0.0504 - last_time_step_mse: 0.0413 - val_loss: 0.0667 - val_last_time_step_mse: 0.0544
Epoch 259/2000
10/10 [==============================] - 0s 47ms/step - loss: 0.0496 - last_time_step_mse: 0.0394 - val_loss: 0.0684 - val_last_time_step_mse: 0.0589
Epoch 260/2000
10/10 [==============================] - 1s 56ms/step - loss: 0.0485 - last_time_step_mse: 0.0410 - val_loss: 0.0666 - val_last_time_step_mse: 0.0590
Epoch 261/2000
10/10 [==============================] - 1s 52ms/step - loss: 0.0495 - last_time_step_mse: 0.0416 - val_loss

In [ ]:
print("Run: "+ str(feature_combinations)+"\n----------------\n" + str(results_list))


In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import keras as keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from itertools import combinations

def show_plot_of_predicted_vs_true(X, y_pred, y_true):
  
  #X_flat = [feature for timestep in X for feature in timestep]

  plt.figure(figsize = (20,8))
  plt.plot(X, marker = '.')
  plt.plot(range(len(X), len(X)+len(y_pred)), y_pred, marker = 'x')
  plt.plot(range(len(X), len(X)+len(y_true)), y_true, marker = '.')

df = pd.read_csv('drive/MyDrive/KDD_Weather_Prediction/madrid.csv')
df['time'] = pd.to_datetime(df['time'])

df = df.sort_values('time')

# Shape definitions
n_steps = 50
y_pred_steps = 10
timesteps_per_batch = n_steps + y_pred_steps

# Calculating the number of batches
n_batches = len(df) // timesteps_per_batch

# Triming the data to a multiple of the batch size
df = df[:n_batches * timesteps_per_batch]

column_names= ["temperature", "wind_speed", "wind_direction"]

#column_names= ["temperature", "wind_speed", "wind_direction", "humidity", "barometric_pressure","solar_radiation", "precipitation"]

results_list = []
counter = 0
for i in range(1, len(column_names)+1):
  for comb in combinations(column_names, i):

    # Droping everything but the temperature
    data = df[list(comb)]

    data = df[['temperature']]

    # Reshaping the data into a 3D array
    data = data.values.reshape((n_batches, timesteps_per_batch, -1))
    # Converting the data to a numpy array
    data = np.array(data)
    

    # Defining split sizes
    train_size = int(n_batches * 0.7)
    validation_size = int(n_batches * 0.15)
    test_size = n_batches - train_size - validation_size

    scaler = StandardScaler()
    num_instances, num_time_steps, num_features = data.shape
    data = np.reshape(data, (-1, num_features))
    data = scaler.fit_transform(data)
    data = np.reshape(data, (num_instances, num_time_steps, num_features))

    # Splitting in train, validation and test set
    X_train = data[:train_size, :n_steps], data[:train_size, -y_pred_steps:, 0]
    X_valid = data[train_size:train_size+validation_size, :n_steps], data[train_size:train_size+validation_size, -y_pred_steps:, 0]
    X_test = data[train_size+validation_size:, :n_steps], data[train_size+validation_size:, -y_pred_steps:, 0]

    print(X_train[0])

    # creating target values
    Y = np.empty((len(data), n_steps, y_pred_steps)) 
    for step_ahead in range(1, y_pred_steps + 1):
      for i in range(len(data)):
        for j in range(n_steps):
          Y[i,j,step_ahead-1] = data[i, j+step_ahead, 0]

    y_train = Y[:train_size]
    y_valid = Y[train_size:train_size + validation_size]
    y_test = Y[train_size+validation_size:]

    print(len(comb))
    model = keras.models.Sequential([
        keras.layers.SimpleRNN(20, name="1", return_sequences=True, input_shape=(None, 1)),
        keras.layers.SimpleRNN(20, name="2",return_sequences=True),
        keras.layers.SimpleRNN(20, name="3",return_sequences=True),
        keras.layers.TimeDistributed(keras.layers.Dense(y_pred_steps))
    ])

    
    def last_time_step_mse(Y_true, Y_pred):
      return keras.metrics.mean_squared_error(Y_true[:,-1], Y_pred[:, -1])

    # Early Stopping Callback
    early_stop = EarlyStopping(monitor='val_loss', patience=200)

    model.compile(loss='mse', optimizer='adam', metrics=[last_time_step_mse])
    model.fit(X_train, y_train, epochs= 100, validation_data=(X_valid, y_valid), callbacks=[early_stop])
    breakpoint()
    y_pred = model.predict(X_test)
    results_list.append(str(comb)+ ": "+ np.array(last_time_step_mse(y_pred, y_test)).mean())
    print(results_list)

[[[-1.18620559]
  [-1.21060748]
  [-1.18620559]
  ...
  [-1.08859804]
  [-1.19840653]
  [-1.22280842]]

 [[-0.61276123]
  [-0.55175651]
  [-0.35654141]
  ...
  [-1.03979426]
  [-0.58835934]
  [-0.50295273]]

 [[-1.38142069]
  [-1.4668273 ]
  [-1.44242541]
  ...
  [-1.01539238]
  [-1.0763971 ]
  [-1.1618037 ]]

 ...

 [[-0.72256972]
  [-0.78357444]
  [-0.74697161]
  ...
  [-0.51515368]
  [-0.62496217]
  [-0.68596689]]

 [[-0.83237822]
  [-0.80797633]
  [-0.69816783]
  ...
  [-0.69816783]
  [-0.51515368]
  [-0.40534518]]

 [[-0.92998577]
  [-1.00319143]
  [-1.06419615]
  ...
  [-0.64936406]
  [-0.69816783]
  [-0.72256972]]]
1
Epoch 1/100


ValueError: ignored

In [5]:
from itertools import combinations
column_names= ["time", "temperature", "wind_speed", "wind_direction", "humidity", "barometric_pressure","solar_radiation", "precipitation"]

counter = 0
for i in range(1, len(column_names)+1):
  for comb in combinations(column_names, i):
    print(list(comb))
    counter += 1
print(counter)


['time']
['temperature']
['wind_speed']
['wind_direction']
['humidity']
['barometric_pressure']
['solar_radiation']
['precipitation']
['time', 'temperature']
['time', 'wind_speed']
['time', 'wind_direction']
['time', 'humidity']
['time', 'barometric_pressure']
['time', 'solar_radiation']
['time', 'precipitation']
['temperature', 'wind_speed']
['temperature', 'wind_direction']
['temperature', 'humidity']
['temperature', 'barometric_pressure']
['temperature', 'solar_radiation']
['temperature', 'precipitation']
['wind_speed', 'wind_direction']
['wind_speed', 'humidity']
['wind_speed', 'barometric_pressure']
['wind_speed', 'solar_radiation']
['wind_speed', 'precipitation']
['wind_direction', 'humidity']
['wind_direction', 'barometric_pressure']
['wind_direction', 'solar_radiation']
['wind_direction', 'precipitation']
['humidity', 'barometric_pressure']
['humidity', 'solar_radiation']
['humidity', 'precipitation']
['barometric_pressure', 'solar_radiation']
['barometric_pressure', 'precipita